In [3]:
import pandas as pd
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop

In [5]:
# Replace 'your_excel_file.xlsx' with the path to your Excel file
file_path = 'train_data.xlsx'

# Load the second sheet into a DataFrame
df = pd.read_excel(file_path, sheet_name=1)  # sheet_name is 0-indexed; 1 refers to the second sheet
# df = pd.concat([df, df[df['PCOS (Y/N)'] == 0]], axis=0)

# Filter out anomaly for cycle irregularity
df = df[df["Cycle(R/I)"] != 5]

In [6]:
num_no_pcos = len(df[df['PCOS (Y/N)'] == 0])
num_has_pcos = len(df[df['PCOS (Y/N)'] == 1])
total = len(df)

print(str(round(float(num_no_pcos) / total * 100, 2)) + "%")
print(num_no_pcos)

67.41%
364


In [10]:
# One hot encoding for blood group and Cycle regularity
from sklearn.preprocessing import OneHotEncoder 

# Create an instance of One-hot-encoder 
encoder = OneHotEncoder() 
  
# Passing encoded columns 
  
blood_group_data = pd.DataFrame(encoder.fit_transform(df[['Blood Group']]).toarray()) 
blood_group_columns = encoder.get_feature_names_out()
print(blood_group_columns)
blood_group_data.columns = blood_group_columns

cycle_ri_data = pd.DataFrame(encoder.fit_transform(df[['Cycle(R/I)']]).toarray()) 
cycle_ri_columns = encoder.get_feature_names_out()
print(cycle_ri_columns)
cycle_ri_data.columns = cycle_ri_columns

# Merge with main 
encoded_df = df.join(blood_group_data).join(cycle_ri_data)
  
print(encoded_df)

['Blood Group_11' 'Blood Group_12' 'Blood Group_13' 'Blood Group_14'
 'Blood Group_15' 'Blood Group_16' 'Blood Group_17' 'Blood Group_18']
['Cycle(R/I)_2' 'Cycle(R/I)_4']
     Sl. No  Patient File No.  PCOS (Y/N)   Age (yrs)  Weight (Kg)  \
0         1                 1           0          28         44.6   
1         2                 2           0          36         65.0   
2         3                 3           1          33         68.8   
3         4                 4           0          37         65.0   
4         5                 5           0          25         52.0   
..      ...               ...         ...         ...          ...   
536     537               537           0          35         50.0   
537     538               538           0          30         63.2   
538     539               539           0          36         54.0   
539     540               540           0          27         50.0   
540     541               541           1          23      

In [11]:
# Filter out unwanted columns
basic_metadata_indices = list(range(2,7))
menstrual_cycle_indices = list(range(12,16))
symptom_indices = list(range(30,37))
encoded_cols_indices = list(range(45,55))

columns_for_filtering = basic_metadata_indices + menstrual_cycle_indices + symptom_indices + encoded_cols_indices

filtered_df = encoded_df.iloc[:,columns_for_filtering]

# Clean NaN values
filtered_df = filtered_df.dropna()

print(filtered_df)

     PCOS (Y/N)   Age (yrs)  Weight (Kg)  Height(Cm)         BMI  \
0             0          28         44.6      152.000  19.300000   
1             0          36         65.0      161.500  24.921163   
2             1          33         68.8      165.000  25.270891   
3             0          37         65.0      148.000  29.674945   
4             0          25         52.0      161.000  20.060954   
..          ...         ...          ...          ...        ...   
535           0          26         80.0      161.544  30.700000   
536           0          35         50.0      164.592  18.500000   
537           0          30         63.2      158.000  25.300000   
538           0          36         54.0      152.000  23.400000   
539           0          27         50.0      150.000  22.200000   

     Cycle length(days)  Marraige Status (Yrs)  Pregnant(Y/N)  \
0                     5                    7.0              0   
1                     5                   11.0       

In [12]:
# Split dataset into train and test

X = filtered_df.iloc[:,1:]
y = filtered_df.iloc[:,0]

In [34]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Split the data and labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3, stratify=y)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
y_train = scaler.fit_transform(y_train.reshape(-1,1))
y_test = scaler.fit_transform(y_test.reshape(-1,1))

num_features = X_train.shape[1]
print(num_features)

25


In [26]:
print(X_train)

[[0.57142857 0.72857143 0.65116279 ... 0.         1.         0.        ]
 [0.42857143 0.36507937 0.18604651 ... 0.         0.         1.        ]
 [0.32142857 0.49206349 0.39534884 ... 0.         1.         0.        ]
 ...
 [0.42857143 0.14285714 0.48837209 ... 0.         1.         0.        ]
 [0.39285714 0.34761905 0.37209302 ... 0.         1.         0.        ]
 [0.17857143 0.41269841 0.25581395 ... 0.         1.         0.        ]]


In [41]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.regularizers import l2

model = Sequential()
model.add(Dense(1, input_dim=X_train.shape[1], activation='sigmoid', kernel_regularizer=l2(0.01)))

model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=10)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")


Epoch 1/100
43/43 [==============================] - 1s 2ms/step - loss: 0.5323 - accuracy: 0.7739
Epoch 2/100
43/43 [==============================] - 0s 2ms/step - loss: 0.4827 - accuracy: 0.8065
Epoch 3/100
43/43 [==============================] - 0s 1ms/step - loss: 0.4614 - accuracy: 0.8252
Epoch 4/100
43/43 [==============================] - 0s 1ms/step - loss: 0.4528 - accuracy: 0.8275
Epoch 5/100
43/43 [==============================] - 0s 1ms/step - loss: 0.4505 - accuracy: 0.8322
Epoch 6/100
43/43 [==============================] - 0s 1ms/step - loss: 0.4465 - accuracy: 0.8275
Epoch 7/100
43/43 [==============================] - 0s 1ms/step - loss: 0.4415 - accuracy: 0.8322
Epoch 8/100
43/43 [==============================] - 0s 1ms/step - loss: 0.4434 - accuracy: 0.8392
Epoch 9/100
43/43 [==============================] - 0s 1ms/step - loss: 0.4416 - accuracy: 0.8322
Epoch 10/100
43/43 [==============================] - 0s 1ms/step - loss: 0.4434 - accuracy: 0.8345
Epoch 11/

In [42]:
model.save('model1.keras')

In [1]:
from keras.models import load_model

loaded_model = load_model('model1.keras')

KeyboardInterrupt: 

In [45]:
# Age, Weight, Height, BMI, Cycle Length, Marriage Status, Pregnant, Number of abortions, Weight gain, Hair growth, Skin Darkening, Hair Loss, Pimples, Fast Food, Regular Exercise, Blood Group[8], Cycle regularity[2]
X_single = [26, 55, 165, 22.7, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]
# X_single = [26, 64, 165, 22.7, 5, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]

X_single_reshaped = np.array(X_single).reshape(1, -1)

# Make a prediction
single_pred = loaded_model.predict(X_single_reshaped)

print(f"The predicted risk of PCOS is: {single_pred[0]}")

1/1 [==============================] - 0s 37ms/step
The predicted risk of PCOS is: [0.2971613]


In [2]:
# Python

import tensorflowjs as tfjs

tfjs.converters.save_keras_model(loaded_model, "./")

2024-02-13 00:24:04.508573: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


NotFoundError: dlopen(/Users/karthikcottur/Imagine Cup/pcos_diagnosis/env/lib/python3.11/site-packages/tensorflow_decision_forests/tensorflow/ops/inference/inference.so, 0x0006): tried: '/Users/karthikcottur/Imagine Cup/pcos_diagnosis/env/lib/python3.11/site-packages/tensorflow_decision_forests/tensorflow/ops/inference/inference.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/Users/karthikcottur/Imagine Cup/pcos_diagnosis/env/lib/python3.11/site-packages/tensorflow_decision_forests/tensorflow/ops/inference/inference.so' (no such file), '/Users/karthikcottur/Imagine Cup/pcos_diagnosis/env/lib/python3.11/site-packages/tensorflow_decision_forests/tensorflow/ops/inference/inference.so' (not a mach-o file)

In [33]:
# Validating with different states

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

for i in range(1,11):
    print("State" + str(i))
    # Split the data and labels into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i, stratify=y)

    X_train = X_train.to_numpy()
    X_test = X_test.to_numpy()
    y_train = y_train.to_numpy()
    y_test = y_test.to_numpy()

    scaler = MinMaxScaler()

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    y_train = scaler.fit_transform(y_train.reshape(-1,1))
    y_test = scaler.fit_transform(y_test.reshape(-1,1))

    num_features = X_train.shape[1]

    model = Sequential()
    model.add(Dense(1, input_dim=X_train.shape[1], activation='sigmoid', kernel_regularizer=l2(0.01)))

    model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0)

    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test loss: {loss}, Test accuracy: {accuracy}")

    model.save('model1.keras')

    loaded_model = load_model('model1.keras')

    # Age, Weight, Height, BMI, Cycle Length, Marriage Status, Pregnant, Number of abortions, Weight gain, Hair growth, Skin Darkening, Hair Loss, Pimples, Fast Food, Regular Exercise, Blood Group[8], Cycle regularity[2]
    X_single_good = [26, 58, 165, 21.3, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]
    X_single_bad = [26, 64, 165, 22.7, 5, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]

    X_single_good_reshaped = np.array(X_single_good).reshape(1, -1)
    X_single_bad_reshaped = np.array(X_single_bad).reshape(1, -1)

    # Make a prediction
    good_pred = loaded_model.predict(X_single_good_reshaped)
    bad_pred = loaded_model.predict(X_single_bad_reshaped)

    print(f"Good Prediction: {good_pred[0]}")
    print(f"Bad Prediction: {bad_pred[0]}")

State1
4/4 [==============================] - 0s 2ms/step - loss: 0.4480 - accuracy: 0.8426
Test loss: 0.44796887040138245, Test accuracy: 0.8425925970077515
1/1 [==============================] - 0s 25ms/step
Good Prediction: [0.924393]
Bad Prediction: [0.9995049]
State2
4/4 [==============================] - 0s 2ms/step - loss: 0.4181 - accuracy: 0.8519
Test loss: 0.4181115925312042, Test accuracy: 0.8518518805503845
1/1 [==============================] - 0s 25ms/step
Good Prediction: [0.9325363]
Bad Prediction: [0.99947685]
State3
4/4 [==============================] - 0s 2ms/step - loss: 0.4329 - accuracy: 0.8333
Test loss: 0.4328779876232147, Test accuracy: 0.8333333134651184
1/1 [==============================] - 0s 27ms/step
Good Prediction: [0.04593318]
Bad Prediction: [0.8635837]
State4
4/4 [==============================] - 0s 2ms/step - loss: 0.4748 - accuracy: 0.8148
Test loss: 0.4748082458972931, Test accuracy: 0.8148148059844971
1/1 [==============================] - 0s 2

In [52]:
from sklearn.preprocessing import OneHotEncoder 

# Create an instance of One-hot-encoder 
encoder = OneHotEncoder() 
  
blood_groups = pd.DataFrame({
    'BloodGroup': [11, 12, 13, 14, 15, 16, 17, 18]
})

encoder = OneHotEncoder(sparse_output=False)
encoder.fit(blood_groups)


new_input_data = np.array([15]).reshape(-1,1)

# Get the feature names for the transformed data
feature_names = encoder.get_feature_names_out()

print(feature_names)

transformed_input_data = encoder.transform(new_input_data)

print(transformed_input_data)

# transformed_input_df = pd.DataFrame(transformed_input_data, columns=feature_names)
  
# print(transformed_input_df)

['BloodGroup_11' 'BloodGroup_12' 'BloodGroup_13' 'BloodGroup_14'
 'BloodGroup_15' 'BloodGroup_16' 'BloodGroup_17' 'BloodGroup_18']
[[0. 0. 0. 0. 1. 0. 0. 0.]]


/Users/karthikcottur/Imagine Cup/pcos_diagnosis/env/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
